In [60]:
import pandas as pd
import cx_Oracle

# Load cleaned dataset
df = pd.read_csv('../data/task2_results.csv')

# Replace with your own credentials and DSN
dsn = cx_Oracle.makedsn("localhost", 1521, service_name="XEPDB1")
conn = cx_Oracle.connect(user="endalew", password="mine", dsn=dsn)
cursor = conn.cursor()

# Insert unique banks into `banks` table and map to IDs
banks = df['bank'].dropna().unique()
bank_id_map = {}

for bank in banks:
    # Check if bank already exists
    cursor.execute("SELECT bank_id FROM banks WHERE bank_name = :1", [bank])
    result = cursor.fetchone()

    if result:
        bank_id_raw = result[0] # Use a temporary variable for the raw value
        print(f"DEBUG: Existing bank_id type: {type(bank_id_raw)}, value: {bank_id_raw}") # Debug print
    else:
        bank_id_var = cursor.var(cx_Oracle.NUMBER)
        cursor.execute(
            "INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
            [bank, bank_id_var]
        )
        bank_id_raw = bank_id_var.getvalue() # Use a temporary variable for the raw value
        print(f"DEBUG: New bank_id type: {type(bank_id_raw)}, value: {bank_id_raw}") # Debug print

    # THIS IS THE CRUCIAL CHANGE: Safely convert to int
    if isinstance(bank_id_raw, list) and len(bank_id_raw) > 0:
        # Extract the first element if it's a list (as your debug showed)
        bank_id_map[bank] = int(bank_id_raw[0])
    else:
        # Otherwise, assume it's directly convertible (e.g., a float or int)
        bank_id_map[bank] = int(bank_id_raw)

# Prepare data for efficient insertion using executemany
print(df.columns)  # See actual column names

# Suppose your date column is 'review_date', then:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (review_text, rating, sentiment_label, sentiment_score, theme, bank_id)
        VALUES (:1, :2, :3, :4, :5, :6)
    """, (
        row['review'], row['rating'], 
        row.get('sentiment_label', None), row.get('sentiment_score', None),
        row.get('theme', None), bank_id_map[row['bank']]
    ))


# Commit and close
conn.commit()
cursor.close()
conn.close()

print("Data successfully inserted into the database.")

DEBUG: New bank_id type: <class 'list'>, value: [51.0]
DEBUG: New bank_id type: <class 'list'>, value: [52.0]
DEBUG: New bank_id type: <class 'list'>, value: [53.0]
Index(['review', 'bank', 'rating', 'sentiment_label', 'sentiment_score',
       'themes'],
      dtype='object')
Data successfully inserted into the database.
